In [25]:
import tensorflow as tf

dataset = tf.keras.utils.image_dataset_from_directory(
    './dataset/valid',
    label_mode='binary',
    image_size=(64, 64),
    batch_size=8,
    shuffle=True
)

Found 3000 files belonging to 2 classes.


In [26]:
from tensorflow.keras import layers, models

def build_discriminator():
    model = models.Sequential([
        layers.Input(shape=(64, 64, 3)),
        layers.Conv2D(64, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(0.2),
        layers.Dropout(0.3),

        layers.Conv2D(128, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(0.2),
        layers.Dropout(0.3),

        layers.Flatten(),
        layers.Dense(1, activation='sigmoid')  # saída binária: 1 (real) ou 0 (fake)
    ])

    return model

# Aqui é Criado e compilado o discriminador
discriminator = build_discriminator()
discriminator.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

print("✅ Discriminador criado!")


✅ Discriminador criado!


In [27]:
from tensorflow.keras import layers, models

def build_generator():
    model = models.Sequential([
        layers.Input(shape=(100,)),  # vetor de ruído com 100 dimensões

        # 1. Projeção e reshape para formar um "bloco inicial"
        layers.Dense(8 * 8 * 256),
        layers.Reshape((8, 8, 256)),

        # 2. Camadas de upsampling (aumentar a imagem)
        layers.Conv2DTranspose(128, kernel_size=4, strides=2, padding='same'),
        layers.LeakyReLU(0.2),

        layers.Conv2DTranspose(64, kernel_size=4, strides=2, padding='same'),
        layers.LeakyReLU(0.2),

        layers.Conv2DTranspose(3, kernel_size=4, strides=2, padding='same', activation='tanh')  # saída final
    ])

    return model

generator = build_generator()
generator.summary()


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                 │ (None, 16384)          │     1,654,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape_3 (Reshape)             │ (None, 8, 8, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_9              │ (None, 16, 16, 128)    │       524,416 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_12 (LeakyReLU)      │ (None, 16, 16, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_10             │ (None, 32, 32, 64)     │       131,136 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_13 (LeakyReLU)      │ (None, 32, 32, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_11             │ (None, 64, 64, 3)      │         3,075 │
│ (Conv2DTranspose)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,313,411 (8.82 MB)

 Trainable params: 2,313,411 (8.82 MB)

 Non-trainable params: 0 (0.00 B)

In [28]:
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Input

# Congela o discriminador ao montar a GAN
discriminator.trainable = False

# Entrada: ruído
gan_input = Input(shape=(100,))
fake_image = generator(gan_input)           # Gerador que transforma ruído em imagem
gan_output = discriminator(fake_image)      # Discriminador faz a  avalião a imagem gerada

# Modelo completo GAN : ruído → imagem gerada → classificação
gan = Model(gan_input, gan_output)
gan.compile(optimizer=Adam(learning_rate=0.0002), loss='binary_crossentropy')

print("✅ GAN criada e compilada!")


✅ GAN criada e compilada!


In [ ]:
import tensorflow as tf

# Carregando o dataset real/fake
dataset = tf.keras.utils.image_dataset_from_directory(
    './dataset/valid',
    label_mode='binary',
    image_size=(64, 64),
    batch_size=4,
    shuffle=True
)


In [29]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

# Quais os Parâmetros
EPOCHS = 20
BATCH_SIZE = 8
LATENT_DIM = 100

# Normalização das imagens
def preprocess_images(dataset):
    return dataset.map(lambda x, y: ((tf.cast(x, tf.float32) / 127.5) - 1.0, y),
                       num_parallel_calls=tf.data.AUTOTUNE).prefetch(tf.data.AUTOTUNE)

dataset_normalizado = preprocess_images(dataset)

# Gerar ruído
def generate_noise(batch_size, dim=100):
    return np.random.normal(0, 1, size=(batch_size, dim))

# Mostrar imagens
def generate_images(generator, epoch):
    noise = generate_noise(16)
    generated_images = generator.predict(noise)
    generated_images = 0.5 * generated_images + 0.5

    fig, axs = plt.subplots(4, 4, figsize=(4, 4))
    count = 0
    for i in range(4):
        for j in range(4):
            axs[i, j].imshow(generated_images[count])
            axs[i, j].axis('off')
            count += 1
    plt.suptitle(f"Imagens Geradas - Época {epoch}")
    plt.show()

# Treinamento
for epoch in range(1, EPOCHS + 1):
    for real_images, _ in dataset_normalizado:
        batch_size = real_images.shape[0]
        real_images = tf.squeeze(real_images)  # remove dimensão extra, se houver

        # Treina o Discriminador
        noise = generate_noise(batch_size)
        fake_images = generator.predict(noise)

        X_combined = np.concatenate([real_images.numpy(), fake_images])
        y_combined = np.concatenate([np.ones((batch_size, 1)), np.zeros((batch_size, 1))])

        discriminator.trainable = True
        d_loss, d_acc = discriminator.train_on_batch(X_combined, y_combined)

        # Treina o Gerador via GAN
        noise = generate_noise(batch_size)
        misleading_labels = np.ones((batch_size, 1))  # o gerador tenta enganar

        discriminator.trainable = False
        g_loss = gan.train_on_batch(noise, misleading_labels)

    if epoch % 10 == 0:

        print(f"Epoch {epoch} - D Loss: {d_loss:.4f}, D Acc: {d_acc:.4f}, G Loss: {g_loss:.4f}")
        generate_images(generator, epoch)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━

KeyboardInterrupt: 

In [31]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
from PIL import Image
import io
import random
import os

CAMINHO_REAL = './dataset/valid/real'
CAMINHO_FAKE = './dataset/valid/fake'
img_usuario = None

# Pré-processamento
def preprocess_image(img):
    img = tf.image.resize(img, (64, 64))
    img = tf.cast(img, tf.float32)
    img = (img / 127.5) - 1.0
    img = tf.expand_dims(img, axis=0)
    return img

# Obter score
def obter_score(img_tensor):
    pred = discriminator.predict(img_tensor)
    return pred[0][0]

# upload
def testar_upload_imagem(change):
    global img_usuario
    try:
        uploaded_file = list(change['new'].values())[0]
        content = uploaded_file['content']
        img = Image.open(io.BytesIO(content)).convert("RGB")
        img_usuario = tf.convert_to_tensor(np.array(img))
        print("📸 Imagem do usuário carregada com sucesso! Agora escolha uma comparação abaixo.")
    except Exception as e:
        print("❌ Erro ao carregar imagem:", e)

# Compara com imagem real
def comparar_com_real(_):
    global img_usuario
    if img_usuario is None:
        print("⚠️ Primeiro faça o upload de uma imagem.")
        return
    try:
        arquivos = os.listdir(CAMINHO_REAL)
        img_path = os.path.join(CAMINHO_REAL, random.choice(arquivos))
        img_real = Image.open(img_path).convert("RGB")
        img_real_tensor = tf.convert_to_tensor(np.array(img_real))

        img_u = preprocess_image(img_usuario)
        img_r = preprocess_image(img_real_tensor)

        score_u = obter_score(img_u)
        score_r = obter_score(img_r)

        plt.figure(figsize=(10, 5))

        plt.subplot(1, 2, 1)
        plt.imshow((img_u[0] + 1.0) / 2.0)
        plt.title(f"Sua imagem\nConfiança: {score_u:.2f}")
        plt.axis('off')

        plt.subplot(1, 2, 2)
        plt.imshow((img_r[0] + 1.0) / 2.0)
        plt.title(f"Imagem REAL\nConfiança: {score_r:.2f}")
        plt.axis('off')

        plt.show()
    except Exception as e:
        print("❌ Erro na comparação:", e)

# Compara com imagem fake
def comparar_com_fake(_):
    global img_usuario
    if img_usuario is None:
        print("⚠️ Primeiro faça o upload de uma imagem.")
        return
    try:
        arquivos = os.listdir(CAMINHO_FAKE)
        img_path = os.path.join(CAMINHO_FAKE, random.choice(arquivos))
        img_fake = Image.open(img_path).convert("RGB")
        img_fake_tensor = tf.convert_to_tensor(np.array(img_fake))

        img_u = preprocess_image(img_usuario)
        img_f = preprocess_image(img_fake_tensor)

        score_u = obter_score(img_u)
        score_f = obter_score(img_f)

        plt.figure(figsize=(10, 5))

        plt.subplot(1, 2, 1)
        plt.imshow((img_u[0] + 1.0) / 2.0)
        plt.title(f"Sua imagem\nConfiança: {score_u:.2f}")
        plt.axis('off')

        plt.subplot(1, 2, 2)
        plt.imshow((img_f[0] + 1.0) / 2.0)
        plt.title(f"Imagem FAKE\nConfiança: {score_f:.2f}")
        plt.axis('off')

        plt.show()
    except Exception as e:
        print("❌ Erro na comparação:", e)

def avaliar_imagem(img):
    img_tensor = preprocess_image(img)
    score = obter_score(img_tensor)

    plt.imshow((img_tensor[0] + 1.0) / 2.0)
    plt.axis('off')
    plt.title("Imagem Avaliada")
    plt.show()

    print(f"🔍 Confiança de ser REAL: {score:.4f}")
    if score > 0.5:
        print("✅ O modelo acredita que é uma IMAGEM REAL")
    else:
        print("❌ O modelo acredita que é uma IMAGEM FALSA")

def testar_aleatoria_real(_):
    arquivos = os.listdir(CAMINHO_REAL)
    img_path = os.path.join(CAMINHO_REAL, random.choice(arquivos))
    img = Image.open(img_path).convert("RGB")
    img = tf.convert_to_tensor(np.array(img))
    avaliar_imagem(img)

def testar_aleatoria_fake(_):
    arquivos = os.listdir(CAMINHO_FAKE)
    img_path = os.path.join(CAMINHO_FAKE, random.choice(arquivos))
    img = Image.open(img_path).convert("RGB")
    img = tf.convert_to_tensor(np.array(img))
    avaliar_imagem(img)

def testar_fake_generator(_):
    noise = generate_noise(1)
    fake_img = generator.predict(noise)
    avaliar_imagem((0.5 * fake_img[0] + 0.5) * 255.0)

# Botões
btn_real = widgets.Button(description="🔍 Testar REAL imagem")
btn_real.on_click(testar_aleatoria_real)

btn_fake_dataset = widgets.Button(description="🧪 Testar FAKE imagem ")
btn_fake_dataset.on_click(testar_aleatoria_fake)

btn_fake_gen = widgets.Button(description="🤖 Testar GERADA imagem")
btn_fake_gen.on_click(testar_fake_generator)

btn_comp_real = widgets.Button(description="📊 Comparar REAL")
btn_comp_real.on_click(comparar_com_real)

btn_comp_fake = widgets.Button(description="📊 Comparar FAKE")
btn_comp_fake.on_click(comparar_com_fake)

upload = widgets.FileUpload(accept='image/*', multiple=False)
upload.observe(testar_upload_imagem, names='value')

# Mostrar interface
display(widgets.HBox([btn_real, btn_fake_dataset, btn_fake_gen]))
print("📂 Faça o upload de uma imagem:")
display(upload)
display(widgets.HBox([btn_comp_real, btn_comp_fake]))


📂 Faça o upload de uma imagem:


FileUpload(value=(), accept='image/*', description='Upload')